In [ ]:
!pip install tensorflow==2.6.0
!pip install keras==2.6.*

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

In [ ]:
tf.__version__

In [ ]:
train = pd.read_csv('../input/cat-in-the-dat-ii/train.csv')
test = pd.read_csv('../input/cat-in-the-dat-ii/test.csv')

In [ ]:
train.shape,test.shape

In [ ]:
train.drop('id',inplace = True,axis = 1)
labels = train['target']
features = train.drop('target',inplace = False,axis = 1)

In [ ]:
for col in features.isnull().sum().index:
    features[col] = features[col].fillna('missing',inplace = False)

In [ ]:
str_cols = features.select_dtypes(include = object).columns

In [ ]:
def make_ds(features,labels):
    X_train,X_test,y_train,y_test = train_test_split(features,labels,test_size = 0.2)
    X_train = {name:tf.constant(value.map(str).to_numpy().reshape(-1,1)) for name,value in dict(X_train).items()}
    X_test = {name:tf.constant(value.map(str).to_numpy().reshape(-1,1)) for name,value in dict(X_test).items()}
    return (X_train,y_train.to_numpy().reshape(-1,1)),(X_test,y_test.to_numpy().reshape(-1,1))

In [ ]:
a,b = make_ds(features,labels)

In [ ]:
def make_lookup(data,col,vocab):
    lookup = layers.StringLookup(vocabulary = vocab)
    return lookup

In [ ]:
inputs = []
outputs = []
for col in str_cols:
    inp = keras.Input(shape = (1,),name = col,dtype = tf.string)
    l = make_lookup(a[0],col,list(features[col].unique()))
    in_dim = len(l.get_vocabulary())
    out_dim = max(len(l.get_vocabulary()),50)
    x = l(inp)
    x = layers.Embedding(input_dim = in_dim,
                        output_dim = out_dim)(x)
    x = layers.Reshape(target_shape = (out_dim,))(x)
    inputs.append(inp)
    outputs.append(x)
x = layers.Concatenate()(outputs)

x = layers.Dense(100,activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100,activation = 'relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(100,activation = 'relu')(x)
x = layers.Dense(1,activation = 'sigmoid')(x)
model = keras.Model(inputs,x)
    

In [ ]:
keras.utils.plot_model(model,show_shapes = True)

In [ ]:
model.compile(loss = keras.losses.BinaryCrossentropy(),optimizer = 'adam',metrics = ['accuracy'])

In [ ]:
model.fit(a[0],a[1])

In [ ]:
model.evaluate(b[0],b[1])

In [ ]:
test_features = test.drop('id',inplace = False,axis = 1)

In [ ]:
for col in test_features.isnull().sum().index:
    test_features[col] = test_features[col].fillna('missing',inplace = False)

In [ ]:
test_features.isnull().sum()

In [ ]:
test_ds = {name:tf.constant(value.map(str).to_numpy().reshape(-1,1)) for name,value in dict(test_features).items()}

In [ ]:
pred = model.predict(test_ds)

In [ ]:
sub = pd.DataFrame(test['id'],columns = ['id'])

In [ ]:
sub['target'] = pd.Series(tf.squeeze(pred))

In [ ]:
sub.to_csv('submission.csv',index = False)